# 畳み込みニューラルネットワーク

In [3]:
import numpy as np
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) #(9, 75)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) #(90, 75)

(9, 75)
(90, 75)


In [4]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T#フィルターの展開
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

In [5]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

In [48]:
from common.layers import *

from collections import OrderedDict
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))#大きさを半分にするプーリングを想定？
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.lastLayer = SoftmaxWithLoss()
        
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
        
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
        
    def gradient(self, x, t):
        self.loss(x, t)
            
        dout = 1
        dout = self.lastLayer.backward(dout)
            
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
                
        grads={}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
            
        return grads
    
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]
        
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [49]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
#from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.300118257851044
=== epoch:1, train acc:0.087, test acc:0.089 ===
train loss:2.2988220359130755
train loss:2.295447318380627
train loss:2.290871043798905
train loss:2.2860536827346083
train loss:2.2719266760963865
train loss:2.2633038365566307
train loss:2.2341930137051484
train loss:2.2517179401726115
train loss:2.1877753944873755
train loss:2.2074775901568087
train loss:2.127452513113011
train loss:2.1284692716878677
train loss:2.12382919172393
train loss:2.040466116000816
train loss:2.0468897290111676
train loss:1.9269709746388517
train loss:1.8657272868753991
train loss:1.8902392271797641
train loss:1.7616097106923783
train loss:1.628726778311443
train loss:1.556419518632294
train loss:1.6172739758860528
train loss:1.478723617552702
train loss:1.3609264977468962
train loss:1.2585286741103408
train loss:1.1753985882579974
train loss:1.1694251853751318
train loss:1.1470417989608281
train loss:0.9489517689931622
train loss:1.1698178415872216
train loss:0.8926890268209867
t

train loss:0.1852560559065738
train loss:0.2062120780353868
train loss:0.18315281011863085
train loss:0.267439825884637
train loss:0.3320055004440776
train loss:0.254927605715719
train loss:0.18830028609523153
train loss:0.2907441147971175
train loss:0.1528988786847688
train loss:0.16175426596271603
train loss:0.14295280249013784
train loss:0.1875808121602347
train loss:0.14053023758297706
train loss:0.16776037845539238
train loss:0.14344512975524043
train loss:0.1937750717411771
train loss:0.2221613613591888
train loss:0.2592061439983357
train loss:0.22910218476904093
train loss:0.17787676335696276
train loss:0.23414442939982671
train loss:0.20668013206944025
train loss:0.16599077418407607
train loss:0.24617379219861696
train loss:0.22483720492689777
train loss:0.3113948139169825
train loss:0.20585500964847056
train loss:0.24731642549730257
train loss:0.24231794466472567
train loss:0.1339605266137326
train loss:0.21114914834890874
train loss:0.1703926980438763
train loss:0.16840253929

train loss:0.1037363878203857
train loss:0.0891398717327097
train loss:0.08915147451808927
train loss:0.15374463688012815
train loss:0.16568665946674485
train loss:0.09396703655393404
train loss:0.059156130451130655
train loss:0.13291499995955125
train loss:0.12279126007706591
train loss:0.05859938763724993
train loss:0.14339631595506386
train loss:0.1587651088288751
train loss:0.10682659853158849
train loss:0.06754509203084381
train loss:0.07609023105268775
train loss:0.1979022198609616
train loss:0.07625675474506606
train loss:0.11268001978534169
train loss:0.08321896030971965
train loss:0.06506257732739211
train loss:0.07343680482096393
train loss:0.13330858438434437
train loss:0.21653473420124314
train loss:0.05912253521869504
train loss:0.092149691167905
train loss:0.08718057001312791
train loss:0.20008736822641596
train loss:0.09553366519272691
train loss:0.07099306542232832
train loss:0.08065915043915255
train loss:0.1701298134261882
train loss:0.21467289867157166
train loss:0.0

train loss:0.03431307099325209
train loss:0.08384914073735877
train loss:0.1627583157667351
train loss:0.040232736390679945
train loss:0.06967233391887012
train loss:0.09885236721704196
train loss:0.11481590994892071
train loss:0.09364225016878472
train loss:0.11467969128195772
train loss:0.08402582276851506
train loss:0.030312675508224647
train loss:0.08123704064270543
train loss:0.033841278523350754
train loss:0.17318770365591887
train loss:0.05576301772634729
train loss:0.042262805222973494
train loss:0.11754815461745398
train loss:0.044366719606962104
train loss:0.13787378470431463
train loss:0.05276390008525953
train loss:0.11318445224845444
train loss:0.1671674477210546
train loss:0.02561681844574399
train loss:0.05895113963160209
train loss:0.08809755933405906
train loss:0.07541005526281713
train loss:0.03376032382735023
train loss:0.07527305858771989
train loss:0.04820216353916572
train loss:0.11635266020961792
train loss:0.12595767249593062
train loss:0.1672130750959089
train 

train loss:0.06528832976470549
train loss:0.06096889307921291
train loss:0.10075710807806977
train loss:0.08144042704022235
train loss:0.049844252765416416
train loss:0.0825830233381219
train loss:0.03672397846030925
train loss:0.10324618639709385
train loss:0.09448093013763076
train loss:0.04349308859413623
train loss:0.1487353068559347
train loss:0.17040150895299283
train loss:0.05213624767520773
train loss:0.031508975803240725
train loss:0.055593835033109676
train loss:0.04759335946415332
train loss:0.04644287576293778
train loss:0.032135233515298595
train loss:0.07751893558436619
train loss:0.056872830861699544
train loss:0.03724488766349428
train loss:0.15198774796929768
train loss:0.04057342689315273
train loss:0.08105333229704381
train loss:0.03418901976176445
train loss:0.08010867388904627
train loss:0.07439392355178308
train loss:0.12767108167051502
train loss:0.06118208998129265
train loss:0.06621808151047284
train loss:0.08285067829974155
train loss:0.08804380641798232
train

...

train loss:0.06472213762359774
train loss:0.11870164256892965
train loss:0.01993548113999873
train loss:0.0317929942119661
train loss:0.02650666165051252
train loss:0.0066349164708342945
train loss:0.013455545297269157
train loss:0.0960122141700418
train loss:0.01321104802946365
train loss:0.03231420012891223
train loss:0.03331176736485646
train loss:0.016445112877869525
train loss:0.03772381185222245
train loss:0.03989675894362166
train loss:0.03750325232830109
train loss:0.007882354279136593
train loss:0.016446379295338667
train loss:0.017607732581215903
train loss:0.009077126586998408
train loss:0.03937773067705591
train loss:0.01806221014368683
train loss:0.01239767088305232
train loss:0.023396963481394594
train loss:0.06851642494445351
train loss:0.02456739473017854
train loss:0.008799089188859886
train loss:0.029978530898829472
train loss:0.017869462258532923
train loss:0.03739510838562492
train loss:0.021079498479751043
train loss:0.007997082182956005
train loss:0.01590934777038

train loss:0.03275045984296547
train loss:0.0361110793151705
train loss:0.014519757319493067
train loss:0.03053767775459583
train loss:0.03470196239521489
train loss:0.03337629289499274
train loss:0.05631366640099508
train loss:0.01325204695050987
train loss:0.01337772355293179
train loss:0.008685327606645692
train loss:0.03076364585451222
train loss:0.07712780824405077
train loss:0.014627312876736126
train loss:0.07058133085776741
train loss:0.06345164672132446
train loss:0.06383370298814194
train loss:0.005504302151280584
train loss:0.005282200297615408
train loss:0.05122406865058414
train loss:0.00940161016914101
=== epoch:5, train acc:0.982, test acc:0.986 ===
train loss:0.05467645475828692
train loss:0.040367018058968515
train loss:0.012442040033686841
train loss:0.028351738165052206
train loss:0.06414216655204984
train loss:0.01888051732656607
train loss:0.01606174998275349
train loss:0.033555906272881256
train loss:0.07762431009654552
train loss:0.06978253045423628
train loss:0.

train loss:0.06772326995398859
train loss:0.022425655662035134
train loss:0.01091953148769212
train loss:0.04175731644072644
train loss:0.03859341707108091
train loss:0.08353490427550955
train loss:0.028030556524832276
train loss:0.006135412082560722
train loss:0.06252125250266637
train loss:0.07462073803311822
train loss:0.014774718355042247
train loss:0.010434840578327753
train loss:0.027432352758243083
train loss:0.044113333266113726
train loss:0.015395576914380068
train loss:0.013841407928539176
train loss:0.03900100730873162
train loss:0.025697380603552908
train loss:0.013628141312308141
train loss:0.030503300453370966
train loss:0.02042639621963207
train loss:0.019745451225736087
train loss:0.03414045488052784
train loss:0.02391077403628676
train loss:0.030313694522137148
train loss:0.034164451588622094
train loss:0.01055452805247105
train loss:0.010748678106120678
train loss:0.01929675713495054
train loss:0.009564109593176854
train loss:0.01304187177084342
train loss:0.047825938

train loss:0.008250314040614866
train loss:0.02244185876494965
train loss:0.05035624870212763
train loss:0.01934217848257812
train loss:0.012297805881249967
train loss:0.032265300598761794
train loss:0.0082110951597341
train loss:0.005126181638588202
train loss:0.015649817594326217
train loss:0.04149787468473061
train loss:0.0357008515370824
train loss:0.017308274924504135
train loss:0.007314690979395759
train loss:0.06310852604359868
train loss:0.048269140142712094
train loss:0.03825885612636295
train loss:0.06856493294261878
train loss:0.11322249694122291
train loss:0.03247606023368574
train loss:0.04923011866922797
train loss:0.018913160876100496
train loss:0.018717483045354234
train loss:0.02304895684792696
train loss:0.049163085407353965
train loss:0.014660604505376906
train loss:0.031796194667962935
train loss:0.005571954465706167
train loss:0.0039380793199086
train loss:0.027068633425785246
train loss:0.01329082024353206
train loss:0.021155635316192654
train loss:0.0437713832356

KeyboardInterrupt: 